In [2]:
import glob
import numpy as np
import pandas as pd
from astropy.time import Time
import cdflib
import datetime
import spacepy.datamodel as dm
from astropy.table import Table

spatial_window = 5
temporal_window = 15


radar_list = glob.glob('../data/external/Madrigal/2018-2021/*/*.hdf5')
radar_list.sort()
# print(len(radar_list))


gf_list = glob.glob('../data/interim/GRACEFO/KBRNE_relative_v1/dat/*')
gf_list.sort()
# print(len(gf_list))
headerlist=['CDF Epoch', 'GPS', 'Latitude', 'Longitude', 'Radius', 'Latitude_QD', 'Longitude_QD','MLT GRACE_1_Position', 'GRACE_2_Position', 'Iono_Corr', 'Distance', 'Relative_Hor_TEC', 'Relative_Ne']

gf_list_datetimes =[]
for gf_file in gf_list:
    gf = pd.read_csv(gf_file, sep='\s+', header=0,index_col=False, names=headerlist)
    gf_list_datetimes.append((Time(gf['CDF Epoch'].values[0], format='cdf_epoch').datetime).strftime('%Y-%m-%d'))

/Users/andyara/opt/anaconda3/envs/gfz_spacepy/lib/python3.8/site-packages/erfa/core.py:154: ErfaWarning: ERFA function "dtf2d" yielded 1 of "dubious year (Note 6)"
  warnings.warn('ERFA function "{}" yielded {}'.format(func_name, wmsg),


In [85]:
i= 69

radar = (Table.read(radar_list[i], path='Data/Table Layout')).to_pandas()
radar_date = datetime.datetime(int(radar.year[0]), int(radar.month[0]), int(radar.day[0])).strftime('%Y-%m-%d')

date, idx_gf, idx_radar = np.intersect1d(gf_list_datetimes,radar_date, return_indices=True)

if len(idx_radar)>0:
    # print(i,date,idx_gf)

    gf_file = gf_list[idx_gf[0]]
    gf = pd.read_csv(gf_file, sep='\s+', header=0,index_col=False, names=headerlist)
    gf['dates'] = (Time(gf['CDF Epoch'].values, format='cdf_epoch')).datetime

    radar_metadata = (Table.read(radar_list[i], path='Metadata/Experiment Parameters')).to_pandas()

    names = np.array([x.decode() for x in radar_metadata['name']])
    values = np.array([x.decode() for x in radar_metadata['value']])

    radar_metadata = pd.DataFrame({'name': names,
                                   'value': values})
    radar_lat = float(radar_metadata[radar_metadata['name'] == 'instrument latitude']['value'].values[0])
    radar_lon = float(radar_metadata[radar_metadata['name'] == 'instrument longitude']['value'].values[0])


    gf = gf[((gf['Latitude'] <= radar_lat + spatial_window) & (gf['Latitude'] >= radar_lat - spatial_window))]
    gf = gf[((gf['Longitude'] <= radar_lon + spatial_window) & (gf['Longitude'] >= radar_lon - spatial_window))]

    if len(gf)>0:
        # print('------')
        # print(i, date, len(gf))
        # print('------')

        gf_pass = gf[gf.diff(axis=0)['dates'] > datetime.timedelta(minutes=temporal_window)]
        # print(len(gf_pass))

        if len(gf_pass) == 0:  # only one passage
            print(i,'0 GF passage ', gf['dates'][gf.index.min()], '->', gf['dates'][gf.index.max()])

            time_window_i = gf['dates'][gf.index.min()] - datetime.timedelta(minutes=temporal_window)
            time_window_f = gf['dates'][gf.index.max()] + datetime.timedelta(minutes=temporal_window)

            dates = []
            for d in iter(range(0,len(radar))):
                dates.append(datetime.datetime(int(radar.year[0]), int(radar.month[0]), int(radar.day[0])))

            radar['dates'] = dates

            radar_select = radar[(radar['dates'] >= time_window_i) & (radar['dates'] <= time_window_f)]

            if len(radar)>0:
                print(i, len(radar))

        if len(gf_pass) == 1: # two passages
            print(i,1 )
#             print(gf_pass)

            pass_idx = gf[gf['dates'] == gf_pass['dates'][gf_pass.index.min()]].index[0]
            print(pass_idx)
            
            # first one
            print('i, 1 GRF passage', gf['dates'][gf.index.min()], '->', gf['dates'][gf.index[np.argwhere(gf.index == pass_idx)[0][0] - 1]])
            
            # define time window
            time_window_i = gf['dates'][gf.index.min()] - datetime.timedelta(minutes=15)
            time_window_f = gf['dates'][gf.index[np.argwhere(gf.index == pass_idx)[0][0] - 1]] + datetime.timedelta(minutes=15)
            
            dates = []
            for d in iter(range(0, len(radar))):
                dates.append(datetime.datetime(int(radar.year[0]), int(radar.month[0]), int(radar.day[0])))
            
            radar['dates'] = dates
            
            radar_select = radar[(radar['dates'] >= time_window_i) & (radar['dates'] <= time_window_f)]
            
            if len(radar) > 0:
                print(i, len(radar))
                
            # second one
            print('i, 2 GRF passage', gf['dates'][gf.index[np.argwhere(gf.index == pass_idx)[0][0]]], '->', gf['dates'][gf.index.max()], '->')
            
            # define time window
            time_window_i = gf['dates'][gf.index[np.argwhere(gf.index == pass_idx)[0][0]]] - datetime.timedelta(minutes=15)
            time_window_f = gf['dates'][gf.index.max()] + datetime.timedelta(minutes=15)
            
            dates = []
            for d in iter(range(0, len(radar))):
                dates.append(datetime.datetime(int(radar.year[0]), int(radar.month[0]), int(radar.day[0])))
            
            radar['dates'] = dates
            
            radar_select = radar[(radar['dates'] >= time_window_i) & (radar['dates'] <= time_window_f)]
            
            if len(radar) > 0:
                print(i, len(radar))


69 1
9729
i, 1 GRF passage 2019-08-13 01:05:32 -> 2019-08-13 01:08:02
69 71


In [62]:
gf_pass['dates'][gf_pass.index.min()]

Timestamp('2019-08-13 13:30:27')

In [66]:
gf[gf['dates'] == gf_pass['dates'][gf_pass.index.min()]].index[0]

9729

In [79]:
np.argwhere(gf.index == pass_idx)[0][0] - 1

31

In [83]:
gf.index[np.argwhere(gf.index == pass_idx)[0][0] - 1]

820

In [3]:


gf = pd.read_csv(gf_list[0], sep='\s+', header=0,index_col=False, names=headerlist)

In [4]:
gf

,CDF Epoch,GPS,Latitude,Longitude,Radius,Latitude_QD,Longitude_QD,MLT GRACE_1_Position,GRACE_2_Position,Iono_Corr,Distance,Relative_Hor_TEC,Relative_Ne
0,63694811097000,1211627115,38.439,4.258,6872560.85,32.236,79.699,11.34,5434899.7,407032.0,4188164.0,5301553.8,392297.1
1,63694811102000,1211627120,38.122,4.246,6872558.00,31.854,79.660,11.34,5458028.0,407605.9,4157916.8,5325611.3,392988.9
2,63694811107000,1211627125,37.804,4.234,6872555.14,31.473,79.622,11.34,5480990.0,408150.6,4127541.8,5349506.9,393651.3
3,63694811112000,1211627130,37.487,4.222,6872552.27,31.092,79.585,11.34,5503785.2,408666.1,4097039.9,5373239.7,394284.3
4,63694811117000,1211627135,37.170,4.210,6872549.40,30.710,79.549,11.34,5526412.8,409152.5,4066412.1,5396809.0,394888.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
9292,63694857557000,1211673575,-29.186,171.429,6870312.31,-35.365,-109.506,12.11,-5879397.1,883978.8,-3444291.0,-5982707.3,903906.8
9293,63694857562000,1211673580,-29.504,171.415,6870298.52,-35.703,-109.423,12.11,-5860209.5,882500.1,-3477184.4,-5964541.8,902621.4
9294,63694857567000,1211673585,-29.822,171.402,6870284.75,-36.041,-109.339,12.12,-5840843.9,880980.4,-3509970.8,-5946194.8,901295.1
9295,63694857572000,1211673590,-30.139,171.388,6870271.00,-36.379,-109.254,12.13,-5821300.9,879419.6,-3542649.1,-5927666.9,899927.8
